In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import librosa
import matplotlib.pyplot as plt

import os
from PIL import Image
from pathlib import Path
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import tensorflow as tf
import librosa.display



In [ ]:
songname = f'../input/gtzan-genre-collection/genres/blues/blues.00000.au'
y, sr = librosa.load(songname, mono=True, duration=2, offset=0)
ps = librosa.feature.melspectrogram(y=y, sr=sr, hop_length=256, n_fft=512, n_mels=128)
ps = librosa.power_to_db(ps**2)
ps.shape

dataset = []
genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 
          'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}

for genre, genre_number in genres.items():
    for filename in os.listdir(f'../input/gtzan-genre-collection/genres/{genre}'):
        songname = f'../input/gtzan-genre-collection/genres/{genre}/{filename}'
        for index in range(14):
            y, sr = librosa.load(songname, mono=True, duration=2, offset=index*2)
            ps = librosa.feature.melspectrogram(y=y, sr=sr, hop_length=256, n_fft=512, n_mels=64)
            ps = librosa.power_to_db(ps**2)
            dataset.append((ps, genre_number))

print(len(dataset))
import random



In [ ]:
random.shuffle(dataset)

train = dataset[:10000]
valid = dataset[10000:12000]
test = dataset[12000:]

X_train, Y_train = zip(*train)
X_valid, Y_valid = zip(*valid)
X_test, Y_test = zip(*test)

# Reshape for CNN input
X_train = np.array([x.reshape((64, 173, 1)) for x in X_train])
X_valid = np.array([x.reshape((64, 173, 1)) for x in X_valid])
X_test = np.array([x.reshape((64, 173, 1)) for x in X_test])

# One-Hot encoding for classes
Y_train = np.array(tf.keras.utils.to_categorical(Y_train, 10))
Y_valid = np.array(tf.keras.utils.to_categorical(Y_valid, 10))
Y_test = np.array(tf.keras.utils.to_categorical(Y_test, 10))



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout, BatchNormalization

lstm_model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    Dropout(0.3),

    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    Dropout(0.3),

    Conv1D(filters=256, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    Dropout(0.3),

    Bidirectional(LSTM(128, return_sequences=True)),
    BatchNormalization(),
    Dropout(0.3),

    Bidirectional(LSTM(64, return_sequences=True)),
    BatchNormalization(),
    Dropout(0.3),

    LSTM(32),
    BatchNormalization(),
    Dropout(0.3),

    Dense(10, activation='softmax')
])

In [ ]:
import tensorflow as tf

gru_model = tf.keras.Sequential([
    tf.keras.layers.GRU(512, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.GRU(256, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.GRU(128, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.GRU(64, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.GRU(32, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.GRU(16, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='softmax')
])




In [ ]:
# Updated CNN Model with Batch Normalization, Dropout, L2 Regularization, and Learning Rate Schedule
cnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 173, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='softmax')
])



In [ ]:
# Compile models
# Use a learning rate schedule to gradually decrease the learning rate
lstm_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.2), loss="categorical_crossentropy", metrics=['accuracy'])
gru_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.2), loss="categorical_crossentropy", metrics=['accuracy'])
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.2), loss="categorical_crossentropy", metrics=['accuracy'])



In [ ]:
# Training models
lstm_history = lstm_model.fit(X_train, Y_train, epochs=100, batch_size=64, validation_data=(X_valid, Y_valid))


In [ ]:
gru_history = gru_model.fit(X_train, Y_train, epochs=200, batch_size=32, validation_data=(X_valid, Y_valid))

In [ ]:
cnn_history = cnn_model.fit(X_train, Y_train, epochs=100, batch_size=64, validation_data=(X_valid, Y_valid))


In [ ]:
# Plotting Results
plt.figure(figsize=(12, 8))
plt.plot(cnn_history.history['accuracy'])
plt.plot(lstm_history.history['accuracy'])
plt.plot(gru_history.history['accuracy'])
plt.legend(['CNN Accuracy', 'LSTM Accuracy', 'GRU Accuracy'])
plt.show()

plt.figure(figsize=(12, 8))
plt.plot(cnn_history.history['val_accuracy'])
plt.plot(lstm_history.history['val_accuracy'])
plt.plot(gru_history.history['val_accuracy'])
plt.legend(['CNN Val Accuracy', 'LSTM Val Accuracy', 'GRU Val Accuracy'])
plt.show()

plt.figure(figsize=(12, 8))
plt.plot(cnn_history.history['loss'])
plt.plot(lstm_history.history['loss'])
plt.plot(gru_history.history['loss'])
plt.legend(['CNN Loss', 'LSTM Loss', 'GRU Loss'])
plt.show()

plt.figure(figsize=(12, 8))
plt.plot(cnn_history.history['val_loss'])
plt.plot(lstm_history.history['val_loss'])
plt.plot(gru_history.history['val_loss'])
plt.legend(['CNN Val Loss', 'LSTM Val Loss', 'GRU Val Loss'])
plt.show()

# Evaluate Models on Test Data
cnn_test_loss, cnn_test_acc = cnn_model.evaluate(X_test, Y_test, verbose=2)
lstm_test_loss, lstm_test_acc = lstm_model.evaluate(X_test, Y_test, verbose=2)
gru_test_loss, gru_test_acc = gru_model.evaluate(X_test, Y_test, verbose=2)

print('CNN Test Accuracy:', cnn_test_acc)
print('LSTM Test Accuracy:', lstm_test_acc)
print('GRU Test Accuracy:', gru_test_acc)

In [ ]:
# Make predictions on the test set
cnn_predictions = cnn_model.predict(X_test)
lstm_predictions = lstm_model.predict(X_test)
gru_predictions = gru_model.predict(X_test)

# Combine predictions using majority voting
ensemble_predictions = np.argmax(cnn_predictions + lstm_predictions, axis=1)

# Calculate accuracy of the ensemble model
ensemble_accuracy = np.mean(ensemble_predictions == np.argmax(Y_test, axis=1))
print(f"Ensemble Model Accuracy: {ensemble_accuracy}")
